# Lab 6. Text Classification with DistilBERT contextual embeddings

In this Lab, we are going to re-use the code from [Lab 4](https://github.com/501Good/tartu-nlp-2020/blob/master/labs/lab4/Lab4_TextClassificationCNN.ipynb). As you may remember, in Lab 4 we looked into text classification with CNN. We used the [ACL IMDB v.1 Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) that contains movie reviews classified as positive and negative. 

In this notebook, you are going to learn how to train a classifier using BERT contextual embeddings and [transformers](https://github.com/huggingface/transformers) library by HuggingFace.

In [1]:
# xzl
%env CUDA_VISIBLE_DEVICES=2

#xzl: cf 
# https://colab.research.google.com/github/501Good/tartu-nlp-2020/blob/master/labs/lab6/Lab6_TransformersClassification.ipynb#scrollTo=yfUd-Wr3t7rO

env: CUDA_VISIBLE_DEVICES=2


In [2]:
# xzl: configuration
is_printing_model = 0
is_testing = 0
is_saving_model = 0
is_reloading_model = 0

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from pathlib import Path
import time
import random

# Check if we are running on a CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We need to install the transformers library first, if we use Google Colab.

In [4]:
!pip3 install transformers

/bin/bash: pip: command not found


From transformers, we need only the model itself, which is `DistilBertForSequenceClassification`; special optimizer that works with it, which is `AdamW`; and a pretrained tokenizer `DistilBertTokenizer` to feed our data correctly into the model. 

For the sake of saving space and performance, we use `DistilBert` in this notebook.

In [5]:
#from transformers import DistilBertForSequenceClassification, AdamW, DistilBertConfig
# xzl 
from transformers import DistilBertForSequenceClassification, DistilBertConfig
from torch.optim import AdamW 

from transformers import DistilBertTokenizer
from transformers import get_linear_schedule_with_warmup

Downloading and unpacking the data.

In [6]:
# xzl-- one time
# !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !mkdir data/
# !tar -xzf aclImdb_v1.tar.gz -C data/

Initializig the tokenizer from the pretrained `distilbert-base-uncased` model.

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', 
                                                do_lower_case=True,
                                                padding_side='right')

In [8]:
print(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today."))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today.")))

['this', 'is', 'the', 'bert', 'token', '##izer', 'that', 'we', "'", 're', 'going', 'to', 'use', 'today', '.']
[2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012]


In [9]:
PAD = '[PAD]'
PAD_ID = 0
DATA_PATH = Path('data') / 'aclImdb'

batch_size = 8 # xzl default 16 OOM
validation_split = .3
shuffle_dataset = True
random_seed = 42

Here, we are going to re-use the `IMDBDataSet` class that we defined in the Lab 4. The only difference is that we get rid of pretrained word embeddings and replace a Spacy tokenizer with our pretrained `DistilBertTokenizer`.

Remember, that in order to use BERT model, our sequence must have the `[CLS]` special token in the beginning and `[SEP]` in the end of the sequence. If we are using `DistilBertTokenizer.encode()` method, we can just specify `add_special_tokens=True` to not bother with it manually.

Another important thing is that the pretrained `distilbert-base-uncased` model only accepts sequences of maximum length of 512 tokens. We set the `max_length` parameter to the `DistilBertTokenizer.encode()` method to trim them accordingly.

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

Since we already return torch tensors padded to have equal lengths in the Dataset, we don't need to define a custom `collate_fn` function as we did in the Lab 4. The default `collate_fn` function from `DataLoader` will handle the collating for us.

In [10]:
class IMDBDataSet(Dataset):
    def __init__(self, pretrain_tokenizer, data_folder='.data', test=False):
        self.tokenizer = pretrain_tokenizer
        self.label_vocab = {'neg': 0, 'pos': 1}
        self.max_len = 512

        if test:
            self.data_folder = data_folder / 'test'
        else:
            self.data_folder = data_folder / 'train'
            
        self.data = []
        
        if self.data_folder.exists():
            self.load()
        else:
            raise ValueError("Data path doesn't exist!")
        
    def load(self):
        for label in ['pos', 'neg']:
            print(f'Reading {label} sentences...')
            p = self.data_folder / label
            for fname in p.glob('*.txt'):
              with open(fname, 'r', encoding='utf-8') as f:
                text = self.tokenizer.encode(f.readline(),
                                             add_special_tokens=True,
                                             max_length=self.max_len,
                                             pad_to_max_length=True,
                                             return_tensors='pt')
                attention_mask = text > 0
                attention_mask = attention_mask.to(torch.uint8)
              torch_label = torch.tensor(self.label_vocab[label], dtype=torch.long)
              self.data.append((text.squeeze(), attention_mask.squeeze(), torch_label))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1], self.data[idx][2]

In [11]:
train_data = IMDBDataSet(tokenizer, DATA_PATH)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Reading pos sentences...


/u/xl6yq/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Reading neg sentences...


In [12]:
test_data = IMDBDataSet(tokenizer, DATA_PATH, test=True)

Reading pos sentences...
Reading neg sentences...


In [13]:
train_data[0]

(tensor([  101,  8475,  5380,  2003,  2125,  2000,  1037,  2919,  2621,  1012,
          4661,  2010,  5798,  2746,  2039,  1010,  2498,  2842,  3504, 10015,
          1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  2034,
          2002,  2038,  2000,  3745,  2010,  2219,  2282,  2007,  2010,  3428,
          2040,  2024,  2183,  2000,  2448,  1037,  2449,  1012,  2027,  2064,
          1005,  1056,  2079,  2009,  1999,  2037,  4734,  2138,  2027,  2123,
          1005,  1056,  2031,  2438,  2686,  1012,  2125,  2000,  1037,  5798,
          2283,  2002,  2069,  4152,  1002,  1020, 19204,  2015,  2096,  2500,
          2131,  1002,  3590,  1010,  1002,  3486,  1010,  1998,  2130,  1002,
          2753,  1012,  2043,  2028,  1997,  2010,  5798,  5329,  3310,  2220,
          1010,  2002,  2069,  4152,  1037,  4638,  2081,  2041,  2005,  1002,
          2340,  1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,
          2183,  2000,  1996,  2924,  2002, 10229,  

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

In [14]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [15]:
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [16]:

# ----------- xzl       load model -------------------------- #

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", # Use the 6-layer BERT model, with an uncased vocab.
    num_labels = len(train_data.label_vocab),  
    output_attentions = False,
    output_hidden_states = False
)

model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
if is_printing_model:
    
    # Get all of the model's parameters as a list of tuples.
    params = list(model.named_parameters())

    print('The DistilBERT model has {:} different named parameters.\n'.format(len(params)))

    print('==== Embedding Layer ====\n')

    for p in params[0:5]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== First Transformer ====\n')

    for p in params[5:21]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== Output Layer ====\n')

    for p in params[-4:]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

epochs = 1
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# Taken from this tutorial: https://github.com/aniruddhachoudhury/BERT-Tutorials/blob/master/Blog%202/BERT_Fine_Tuning_Sentence_Classification.ipynb
# The code was modified

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

loss_values = []

# xzl
loss_threshold = .6     # higher -> skip more; lower -> skip less
skip_counter = 0

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    #model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed_mins, elapsed_secs = epoch_time(t0, time.time())
            
            # Report progress.
            print(f'  Batch {step:>5,}  of  {len(train_loader):>5,}.    Elapsed: {elapsed_mins:}m {elapsed_secs:}s.')

        # Unpack this training batch from our dataloader. 
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # xzl:
        model.eval()
        
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        
        if loss.item() < loss_threshold: 
            skip_counter += 1
            optimizer.zero_grad()   # just in case ...
            continue
        
        # xzl: worth backprop ... do it again
        model.train()
    
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        loss = outputs[0]
        
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # --- xzl: skip this?? --
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print(f"  Average training loss: {avg_train_loss:.2f}")
    print("  Training epcoh took: {:}m {:}s".format(*epoch_time(t0, time.time())))
    print("  skipped: ", skip_counter, f"{100 * skip_counter/len(train_loader):.2f}%", "loss_threshold", loss_threshold)
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}m {:}s".format(*epoch_time(t0, time.time())))

print("")
print("Training complete!")

In [22]:
if is_testing:
    print("")
    print("Running Testing...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    test_loss, test_accuracy = 0, 0
    nb_test_steps, nb_test_examples = 0, 0

    # Evaluate data for one epoch
    for batch in test_loader:

        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_test_accuracy = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        test_accuracy += tmp_test_accuracy

        # Track the number of batches
        nb_test_steps += 1

    # Report the final accuracy for this test run.
    print("  Accuracy: {0:.2f}".format(test_accuracy/nb_test_steps))
    print("  Testing took: {:}m {:}s".format(*epoch_time(t0, time.time())))

In [23]:
if is_saving_model:
    
    import os

    # Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

    output_dir = './model_save/'

    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Saving model to %s" % output_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Good practice: save your training arguments together with the trained model
    # torch.save(args, os.path.join(output_dir, 'training_args.bin'))

In [24]:
if is_reloading_model:
    # Load a trained model and vocabulary that you have fine-tuned
    model = DistilBertForSequenceClassification.from_pretrained(output_dir)
    tokenizer = DistilBertTokenizer.from_pretrained(output_dir)

    # Copy the model to the GPU.
    model.to(device)